# Implementación de *Simulated Annealing* para el problema del agente viajero

**Importar las librerías necesarias para poder trabajar con los datos y poner un 'seed'**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Leer los datos (la matriz de distancias) y crear un DataFrame a partir de ellos**

In [2]:
CITIES = [ i for i in range(0, 128)]
DATA = pd.read_csv('cities.csv', header=None)

# Vamos a imprimir los primeros 5 para asegurarnos de que todo está bien
DATA.head(5)

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0,966,1513,2964,1149,927,1611,1510,390,466,...,1018,168,565,1700,1636,2019,1458,1564,2871,348
1,966,0,2410,1520,1817,729,686,290,1823,168,...,548,1127,432,2265,558,571,943,198,1917,2541
2,1513,2410,0,604,481,2742,1833,826,214,1618,...,752,486,590,2132,1095,2154,1211,2217,2673,1570
3,2964,1520,604,0,595,1289,1446,466,1139,430,...,1861,861,499,1408,986,2719,1437,769,1038,2343
4,1149,1817,481,595,0,494,550,2641,765,934,...,216,1994,324,2187,260,2586,1974,2352,2243,691


**Hacer un primer camino aleatorio**

In [3]:
PATH = np.random.permutation(CITIES)

**Definir la función del costo del camino actual**

In [4]:
def cost(path: np.array) -> float:
    """Calcula el costo de la ruta elegida al momento

    Args:
        path (np.array): Array que contiene la ruta que se debe de seguir al momento

    Returns:
        float: El costo de la ruta
    """
    global DATA
    
    cost = 0
    
    # Calcular el costo del camino
    for i in range(len(path) - 1):
        cost += DATA.iloc[path[i], path[i + 1]]
        
    # Sumar el costo de volver al principio
    cost += DATA.iloc[path[-1], path[0]]
    return cost

**Definir la función de obtener un nuevo camino 'vecino' del actual**

In [5]:
def generate_path(path: np.array) -> np.array:
    """Genera una ruta tomando dos elementos aleatorios de la ruta, y haciendo un 'reverse' de todos los elementos 
        dentro del array formado por estas dos ciudades como extremos

    Args:
        path (np.array): La ruta actual

    Returns:
        np.array: La nueva ruta
    """
    # Tomar dos elementos aleatorios
    i, j = np.random.choice(path, size=2, replace=False)
    
    # Hacer el swap
    path[i : j] = path[i : j][::-1]
    
    return path

## Función que usa todas las funciones anteriores para generar la mejor ruta

In [6]:
def simulated_annealing(path: np.array, t: float, l: int, max_repeated: int = 10000, max_loops: int = 1000000, t_update: float = 0.99) -> list:
    """Función que implementa el algoritmo de Simulated Annealing

    Args:
        path (np.array): La ruta actual
        t (float): Temperatura
        l (int): Cada cuantas iteraciones se debe de actualizar la temperatura
        max_repeated (int, optional): Cantidad máxima de iteraciones sin mejora. Defaults to 100000.
        max_loops (int, optional): Cantidad máxima de iteraciones. Defaults to 1000000.
        t_update (float, optional): Factor de actualización de la temperatura. Defaults to 0.99.

    Returns:
        np.array: La nueva ruta
    """
    number_of_repeated_differences = 0
    last_diff = 0
    diff = 0
    k = 0
    cost_path = cost(path)
    results = []

    while k < max_loops:
        # Generar una nueva ruta
        new_path = generate_path(path)

        # Calcular la diferencia de costos
        last_diff = diff
        cost_new_path = cost(new_path)
        diff = cost_new_path - cost_path

        # Si la diferencia es menor que 0, entonces la nueva ruta es mejor, por lo que la actual se debe de actualizar
        if diff < 0:
            path = new_path
            cost_path = cost_new_path

        # Si la diferencia es mayor que 0, entonces la nueva ruta es peor, por lo que se debe de probar si debe de ser aceptada
        elif np.random.rand() < np.exp(-diff / t):
            path = new_path
            cost_path = cost_new_path

        # Aumentar K y cada L pasos disminuir T
        k += 1
        if k % l == 0:
            t *= t_update
            
        # Escribir el costo dentro del array de resultados
        results.append(cost_path)

        # Checar si la diferencia es igual a la ultima diferencia, si van varias salirse del ciclo
        if last_diff == diff:
            number_of_repeated_differences += 1

            if number_of_repeated_differences > max_repeated:
                break
            
        else:
            number_of_repeated_differences = 0
            
    return results
    
    

**Ejecutar el programa y graficar el costo**

In [9]:
for T in np.arange(1000, 10000, 1000):
    # create empty array to store results
    L = 5

    # run algorithm
    results = simulated_annealing(path=PATH, t=T, l=L, max_repeated=1000, max_loops=20000, t_update=0.99)

    # graph results
    plt.plot(range(len(results)), results)
    plt.title(f'Cost of Simulated Annealing with T={T}')
    plt.xlabel('k iterations')
    plt.ylabel('Cost')
    plt.show()


    print(f"El camino más corto es de: {results[-1]} unidades")